In [5]:
import numpy as np
import random
from copy import deepcopy

In [6]:
GRID_SIZE = 5
PUNISHMENT_MINUS = [1,1,2,2,2,3,3]

In [7]:
COLOUR_DICT = {
    0: "One", 1: "Blue", 2: "Dark Blue", 3: "Light Blue", 4: "Red", 5: "Orange"
}

In [181]:
# Maybe we can define the grid as an array of tuples, 1st argument indicating whether it is filled, and the 2nd, indicating the type of colour
INITIAL_GRID = [
    [[0, 1], [0, 5], [0, 4], [0, 2], [0, 3]],
    [[0, 3], [0, 1], [0, 5], [0, 4], [0, 2]],
    [[0, 2], [0, 3], [0, 1], [0, 5], [0, 4]],
    [[0, 4], [0, 2], [0, 3], [0, 1], [0, 5]],
    [[0, 5], [0, 4], [0, 2], [0, 3], [0, 1]]
]

# We can also define the tiles that are filled to place gems in the grid, zeros are used here to indicate that nothing is currently present
INITIAL_TILES = [
    [0], [0,0], [0,0,0], [0,0,0,0], [0,0,0,0,0]
]

In [182]:
# Next, let's make a generator to randomly produce assortments of the tops within the game
# That is, we want to produce 7 4-tuples with varying distributions of colour and this MUST be randomized (There are 100 tiles in total and 20 of each colour)
COLOUR_DICT

{0: 'One', 1: 'Blue', 2: 'Dark Blue', 3: 'Light Blue', 4: 'Red', 5: 'Orange'}

In [183]:
def generate_distributions():
    
    while True: 
        curr_list = []
        for i in range(7):
            assort_tuple = (random.randint(1,5), random.randint(1,5), random.randint(1,5), random.randint(1,5))
            curr_list.append(assort_tuple)
        
        if verify_distribution(curr_list):
            return curr_list

def verify_distribution(nums):
    curr_dict = {}
    for tups in nums:
        for x in tups:
            if x in curr_dict:
                curr_dict[x] += 1
            else:
                curr_dict[x] = 1

    for x in curr_dict:
        if curr_dict[x] > 20:
            return False
        
    return True

def convert_nums_to_colours(nums):
    new_list = []
    for x,y,z,w in nums:
        new_list.append((COLOUR_DICT[x], COLOUR_DICT[y], COLOUR_DICT[x], COLOUR_DICT[w]))
    
    return new_list

In [184]:
random_dist = generate_distributions()
random_dist_colors = convert_nums_to_colours(random_dist)

random_dist_colors

[('Light Blue', 'Red', 'Light Blue', 'Red'),
 ('Red', 'Red', 'Red', 'Dark Blue'),
 ('Red', 'Orange', 'Red', 'Orange'),
 ('Light Blue', 'Dark Blue', 'Light Blue', 'Dark Blue'),
 ('Blue', 'Orange', 'Blue', 'Light Blue'),
 ('Orange', 'Light Blue', 'Orange', 'Light Blue'),
 ('Orange', 'Red', 'Orange', 'Light Blue')]

In [185]:
# Remember here that we have to pick A COLOUR amongst the tiles which are present on the top, and the rest of the colors are shoved within the middle
# Need to keep a track of those colours in the middle

In [186]:
# 0 represents the ONE tile
MIDDLE_TILES = [0]

In [187]:
# For predictions, we want an algorithm that will tell the user which of the 7 tiles are best, and of those colours within the tiles, which should be selected.
# Should we simulate choosing each of the tiles and test for the possible points which can be obtained and see which selection maxes the points? 
# This would require simulations of the game
# Let's make a player class for this to represent the information that each player will have

In [453]:
class Player:
    def __init__(self):
        self.grid = deepcopy(INITIAL_GRID)
        self.tiles = deepcopy(INITIAL_TILES)
        self.score = 0
        self.punishment = []
    
    # This function tells the user which row is best to fill out when given a certain number of tiles
    def best_tile_row(self, tiles):
        num_tiles = len(tiles)
        tile_type = tiles[0]
        
        # First: Get the rows in which tiles of the type have not been filed out yet
        avail_rows = []
        for i in range(GRID_SIZE):
            row = self.grid[i]
            for j in range(GRID_SIZE):
                cell = row[j]
                if not cell[0] and cell[1] == tile_type:
                    avail_rows.append(i)
                    break
        
        # Next: are there any rows that are (Empty OR same type tile) AND (same number of slots as number of tiles)
        valid_rows = []
        for row_num in avail_rows:
            tile_row = self.tiles[row_num]
            reduced_tile_row = sorted(list(set(tile_row)))
            
            num_empty = tile_row.count(0)
            
            if num_empty <= num_tiles and ((len(reduced_tile_row) == 1) or (reduced_tile_row[1] == tile_type)):
                valid_rows.append(row_num)
        
        # If there are rows that we can fill then we go ahead and fill them and calculate the best score attainable
        max_score = -999
        most_leftover = 0
        
        if valid_rows:
            best_row = self.tiles[valid_rows[0]]
            for row_num in valid_rows:
                row = self.grid[row_num]
                column_num = 0
                for cell in row:
                    if cell[1] == tile_type:
                        break
                    column_num += 1

                copy_grid = deepcopy(self.grid)
                copy_grid[row_num][column_num][0] = 1

                score = self.get_score(copy_grid, row_num, column_num)

                num_spaces = self.tiles[row_num].count(0)
                leftover = num_tiles - num_spaces

                # Calculate negative score if leftover
                minus = 0
                if leftover:
                    minus = self.punish(leftover)

                score += minus
                if (score > max_score):
                    max_score = score 
                    best_row = self.tiles[row_num]
                    most_leftover = leftover
                    
            for i in range(most_leftover):
                self.punishment.append(0)
            
            num_spaces = best_row.count(0)
            for i in range(len(best_row)):
                best_row[i] = tile_type

            if max_score >= 0:
                return max_score, best_row, most_leftover

        if avail_rows:
            # Get number of tiles in each available row
            for row_num in avail_rows:
                tile_row = self.tiles[row_num]
                reduced_tile_row = sorted(list(set(tile_row))) # Get the existing tile types within
                if len(reduced_tile_row) > 1:
                    reduced_tile_type = reduced_tile_row[1]

                    if (tile_type == reduced_tile_type):
                        # Count the number of empty spaces 

                        num_spaces = tile_row.count(0)
                        num_filled = tile_row.count(tile_type)

                        # Try to fill up the leftover spaces
                        if num_tiles < num_spaces:
                            self.tiles[row_num] = [tile_type] * (num_filled + num_tiles)

                            return 0, tile_row, 0


                else:
                    num_spaces = tile_row.count(0)
                    if num_tiles < num_spaces:
                        for i in range(num_tiles):
                            self.tiles[row_num][i] = tile_type

                        return 0, tile_row, num_tiles

            # If we reached here it means that we could slot in anywhere, we have to invoke punishment to score
            minus = self.punish(num_tiles)        
            for i in range(num_tiles):
                self.punishment.append(0)

            return minus, [], num_tiles

        else:
            # Suffer a penalty
            minus = self.punish(num_tiles)  
            for i in range(num_tiles):
                self.punishment.append(0)

            return minus, [], num_tiles
                
    # The row and column is to tell where the most recent tile has been added so the score can be calculated accordingly
    def get_score(self, grid, row, column):
        
        # Let us check column wise and row wise, how the score will be calculated
        # We go from the tile itself, to the left and check how many consecutively and then to the right consecutively and minus one, same goes for column
        
        score_row = grid[row]
        left_score = 0
        right_score = 0
        for i in range(column-1, -1, -1):
            if not score_row[i][0]:
                break
            left_score += 1
                
        for i in range(column+1, 5):
            if not score_row[i][0]:
                break
            right_score += 1
        
        row_score = left_score + right_score
        
        up_score = 0
        down_score = 0
        score_column = [r[column] for r in grid]
        
        for i in range(row-1, -1, -1):
            if not score_column[i][0]:
                break
            up_score += 1
                
        for i in range(row+1, 5):
            if not score_column[i][0]:
                break
            down_score += 1
        
        column_score = up_score + down_score
        
        total_score = row_score + column_score
        if total_score == 0:
            total_score = 1
            
        return total_score
    
    # This function does not make any changes to the tops nor the number of tiles themselves, it only returns the chosen number of tiles on the selected top
    # The numbers will only be changed after the best possibly outcome is calculated and selected
    def take_colors(self, middles, tiles_arr, tile_colour, TAKE_MIDDLES = False):
        if not TAKE_MIDDLES:
            chosen_tiles = [tile for tile in tiles_arr if tile == tile_colour]
            return chosen_tiles
        
        else:
            chosen_tiles = [tile for tile in middles if tile == tile_colour]
            if 0 in middles:
                return chosen_tiles, True
                
            return chosen_tiles, False


    
    def punish(self, leftover):
        
        pun_len = len(self.punishment)
        minus = 0
        for i in range(leftover):
            punish_index = i + pun_len
            if punish_index >= len(PUNISHMENT_MINUS):
                minus -= 3
            else:
                minus -= PUNISHMENT_MINUS[punish_index]
        
        return minus
    
    def copy(self):
        new_player = Player()
        new_player.tiles = deepcopy(self.tiles)
        new_player.grid = deepcopy(self.grid)
        new_player.punishment = self.punishment.copy()
        new_player.score = self.score
        
        return new_player
        
        

In [467]:
# Generate the random starting distribution of tiles and begi

test_player1 = Player()
test_player2 = Player()
test_player3 = Player()
player_arr = [test_player1, test_player2, test_player3]
starting_tiles = generate_distributions()
middle_tiles = [0]

In [468]:
test_player1.tiles

[[0], [0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0]]

In [469]:
# With the appropriate functions we can try to simulate what a round of azul might look like, we can take an approach that maximizes our points and minimizing the enemy's points
# Every time a circle of turns pass, we can update the number of tiles left available and repeat the calculations, since it is not set in stone for what the others might do

In [497]:
# Lets perform a test run calculation

def generate_options(tiles, middle_tiles, player):
    options_arr = []
    unique_middle_colors = list(set(middle_tiles))
    
    if (0 in unique_middle_colors and len(unique_middle_colors) > 1) or (0 not in unique_middle_colors):
        for color in unique_middle_colors:
            if color != 0:
                copy_player_options = player.copy()
                chosen, middle_first = copy_player_options.take_colors(middle_tiles, [], color, TAKE_MIDDLES=True)
                
                points_earned, best_row, leftovers = copy_player_options.best_tile_row(chosen)         
                
                if middle_first:
                    best_row.append(0)
                
                score_tuple = (points_earned, best_row, color, chosen, leftovers)

                options_arr.append(score_tuple)
        
    
    for top in tiles:
        unique_colors = list(set(top))
        
        for color in unique_colors:
            copy_player_options = player.copy()
            chosen = copy_player_options.take_colors(middle_tiles, top, color, TAKE_MIDDLES=False)
        
            # For these chosen tiles, now fill in the tiles and get the expected score
            points_earned, best_row, leftovers = copy_player_options.best_tile_row(chosen)
            score_tuple = (points_earned, best_row, color, top, leftovers)

            options_arr.append(score_tuple)
                
    return options_arr
    
options_arr = generate_options(starting_tiles, middle_tiles, test_player1)
options_arr

[(1, [2], 2, [2], 0),
 (1, [3, 3, 3], 3, [3, 3, 3], 0),
 (1, [1, 1], 1, [1, 1, 2, 4], 0),
 (1, [2], 2, [1, 1, 2, 4], 0),
 (1, [4], 4, [1, 1, 2, 4], 0)]

In [498]:
test_player = Player()
generate_options([[2, 2, 3, 1], [4, 4, 2, 1]], [0, 3, 3, 4], test_player)

[(1, [3, 3, 0], 3, [3, 3], 0),
 (1, [4, 0], 4, [4], 0),
 (1, [1], 1, [2, 2, 3, 1], 0),
 (1, [2, 2], 2, [2, 2, 3, 1], 0),
 (1, [3], 3, [2, 2, 3, 1], 0),
 (1, [1], 1, [4, 4, 2, 1], 0),
 (1, [2], 2, [4, 4, 2, 1], 0),
 (1, [4, 4], 4, [4, 4, 2, 1], 0)]

In [507]:
# Now we can compute a tree of possibilities
# Go through the cycle of players until each possibility has been explored

curr_player_index = 0
middle_tiles = [0]

starting_tiles = generate_distributions()

# Start off with one player, and produce the possible options
# Go through each option and when they select one option, then go to the next player and have repeat

def play_round(players, starting_tiles, middle_tiles, player_index):
    
    if not starting_tiles and not middle_tiles:
        print(players[0].score)
        print(players[0].grid)
        print(players[0].tiles)
    
        print("End\n")
        return
    
    possible_options = generate_options(starting_tiles, middle_tiles, players[player_index])
    
    for option in possible_options:
        copy_player = players[player_index].copy()
        starting_tiles_copy = deepcopy(starting_tiles)
        middle_tiles_copy = middle_tiles.copy()
        points, filled_row, tile_type, chosen_tiles, leftovers = option
        
        if 0 in filled_row:
            filled_row.remove(0)
            copy_player.punishment.append(0)
        
        for i in range(leftovers):
            copy_player.punishment.append(0)
        
        copy_player.score += points
        chose_row_num = len(filled_row) - 1
        
        #Fill The Row for the player
        if filled_row:
            copy_player.tiles[chose_row_num] = filled_row
        
        # Fill in players grid? Perhaps we might not need to ( I think )
        #print(f"For Player {player_index}, Score: {copy_player.score} Tiles: {copy_player.tiles}")
        
        players[player_index] = copy_player
        player_index_next = (player_index + 1) % 3
        
        # Remove tile top from initial list of tiles
        if chosen_tiles in starting_tiles_copy:
            starting_tiles_copy.remove(chosen_tiles)
            
            # Add the unused tiles to the middle
            for tile in chosen_tiles:
                if tile != tile_type:
                    middle_tiles_copy.append(tile)
        else:
            # If tile top not present then remove the tiles from the middle tile list
            if 0 in middle_tiles_copy:
                middle_tiles_copy.remove(0)
            
            while tile_type in middle_tiles_copy:
                middle_tiles_copy.remove(tile_type)
                
        play_round(players, starting_tiles_copy, middle_tiles_copy, player_index_next)
        

In [513]:
player1 = Player()
player2 = Player()
player3 = Player()

In [514]:
players = [player1, player2, player3]
start_index = 0
starting_tiles = [[1,3,3,4], [2,2,3,1], [4,4,2,1]]
middle_tiles = [0]

In [ ]:
play_round(players, starting_tiles, middle_tiles, start_index)

1
[[[0, 1], [0, 5], [0, 4], [0, 2], [0, 3]], [[0, 3], [0, 1], [0, 5], [0, 4], [0, 2]], [[0, 2], [0, 3], [0, 1], [0, 5], [0, 4]], [[0, 4], [0, 2], [0, 3], [0, 1], [0, 5]], [[0, 5], [0, 4], [0, 2], [0, 3], [0, 1]]]
[[1], [0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0]]
End

1
[[[0, 1], [0, 5], [0, 4], [0, 2], [0, 3]], [[0, 3], [0, 1], [0, 5], [0, 4], [0, 2]], [[0, 2], [0, 3], [0, 1], [0, 5], [0, 4]], [[0, 4], [0, 2], [0, 3], [0, 1], [0, 5]], [[0, 5], [0, 4], [0, 2], [0, 3], [0, 1]]]
[[1], [0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0]]
End

1
[[[0, 1], [0, 5], [0, 4], [0, 2], [0, 3]], [[0, 3], [0, 1], [0, 5], [0, 4], [0, 2]], [[0, 2], [0, 3], [0, 1], [0, 5], [0, 4]], [[0, 4], [0, 2], [0, 3], [0, 1], [0, 5]], [[0, 5], [0, 4], [0, 2], [0, 3], [0, 1]]]
[[2], [0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0]]
End

1
[[[0, 1], [0, 5], [0, 4], [0, 2], [0, 3]], [[0, 3], [0, 1], [0, 5], [0, 4], [0, 2]], [[0, 2], [0, 3], [0, 1], [0, 5], [0, 4]], [[0, 4], [0, 2], [0, 3], [0, 1], [0, 5]], [[0, 5]